Importing the needed packages


In [1]:
import matplotlib.pyplot as plt
import numpy as np
import ipywidgets as widgets
from IPython.display import display
import pyvista as pv
import scipy.constants as consts

Getting the coax cable parameters from the user
Plotting the coax cable using PyVista to give the user an intuition for the system

In [7]:
params = [0.1, 1.0, 5.0] # inner radius a, outer radius b, cable length h
N = 100                  # number of faces to display on the cable render
surf = None              # pyvista mesh


coax_widgets = (
    widgets.FloatSlider(min=0.01, max=10.0, step=0.01, value=params[0],
                        description="Inner radius 'a'"),
    widgets.FloatSlider(min=0.01, max=10.0, step=0.01, value=params[1],
                        description="Outer radius 'b'"),
    widgets.FloatSlider(min=0,    max=10.0, step=0.01, value=params[2],
                        description="Cable length 'h'")
)


def create_surf():
    """
    Initializes the global surf with a mesh that describes the coax cable
    """
    global surf
    theta_values = np.linspace(0, 2 * np.pi, N+1)[:-1]
    x = np.cos(theta_values).reshape(1, -1)
    y = np.sin(theta_values).reshape(1, -1)
    x_arr, y_arr = [], []
    for i in [0, 1]:
        x_arr.append(x.copy() * params[i])
        y_arr.append(y.copy() * params[i])
    pts = np.concatenate(
        (
            np.concatenate((x_arr[0], y_arr[0], np.zeros((1, N)))),
            np.concatenate((x_arr[0], y_arr[0], np.full((1, N), params[2]))),
            np.concatenate((x_arr[1], y_arr[1], np.zeros((1, N)))),
            np.concatenate((x_arr[1], y_arr[1], np.full((1, N), params[2])))
        ), 1
    )

    # building the edges of the cylinders
    faces = []
    for i in range(N):
        j = (i+1) % N
        faces.append(np.array([4, i        , j        , j + N    , i + N    ]))
        faces.append(np.array([4, i + 2 * N, j + 2 * N, j + 3 * N, i + 3 * N]))

    # front and back faces of the inner conductor
    faces.append([100] + [i for i in range(0  , 100)])
    faces.append([100] + [i for i in range(100, 200)])
    faces = np.hstack(faces)
    surf = pv.PolyData(pts.transpose(), faces)


def update_display(change):
    global plotter
    if change.name == "value":
        params[change.owner.local_id] = change.new
        create_surf()

for i, e in enumerate(coax_widgets):
    e.local_id = i
    e.observe(update_display)
    display(e)

FloatSlider(value=0.1, description="Inner radius 'a'", max=10.0, min=0.01, step=0.01)

FloatSlider(value=1.0, description="Outer radius 'b'", max=10.0, min=0.01, step=0.01)

FloatSlider(value=5.0, description="Cable length 'h'", max=10.0, step=0.01)

Create/Update the viewport with the newly selected values

Re-run this cell to update the cable render with the newly selected parameters

In [11]:
if params[1] <= params[0]:  # this line is equivalent to - if b <= a
    raise ValueError("Invalid parameter selection:\n\tThe inner radius must be strictly less than the outer")

plotter = pv.PlotterITK()
create_surf()
plotter.add_mesh(surf)
plotter.show()

ValueError: Invalid parameter selection:
	The inner radius must be strictly less than the outer

Computing the capacitance and inductance of the coaxial cables using the equations derived in the report

$L_{in} = \frac{\mu_0h}{8\pi}$

$L_{ext} = \frac{\mu_0h}{2\pi}ln\left(\frac{b}{a}\right)$

$L = L_{in} + L_{ext} = \frac{h}{2\pi}\left(\frac{\mu}{4} + \mu_0ln\left(\frac{b}{a}\right)\right)$

$C=\frac{2\pi h\varepsilon_0}{ln\left(\frac{b}{a}\right)}$

In [4]:
def internal_inductance(h):
    return (consts.mu_0 * h) / (8 * np.pi)

def external_inductance(a, b, h):
    return (consts.mu_0 * h) / (2 * np.pi) * np.log(b / a)

def total_inductance(a, b, h):
    return internal_inductance(h) + external_inductance(a, b, h)

def capacitance(a, b, h):
    return (2 * np.pi * h * consts.epsilon_0) / np.log(b / a)

# Each of the attributes of the cable can be computed using their respective equation:
#     L_in = internal_inductance(params[2])
#     L_ext = external_inductance(*params)
#     L_tot = total_inductance(*params) # redundant function call, L_in + L_ext also works
#     C = capacitance(*params)

Computing each of the cable attributes for a series of b/a values

In [ ]:
# todo: compute the data matrices